In [ ]:
# Cài đặt thư viện đọc video nhanh
!pip install decord

import torch
import torch.nn as nn
import numpy as np
import os
import sys
from pathlib import Path
from tqdm import tqdm
from decord import VideoReader, cpu
import torchvision.transforms as transforms
import warnings

# Tắt các cảnh báo không cần thiết
warnings.filterwarnings('ignore')

# --- THAY ĐỔI QUAN TRỌNG ---
# Thêm đường dẫn tới dataset chứa file .py của BẠN
# !!! Sửa 'my-i3d-files' thành tên dataset bạn đã tạo ở Bước 1
I3D_SOURCE_DIR = "/kaggle/input/my-i3d-files" 
sys.path.append(I3D_SOURCE_DIR)

try:
    from pytorch_i3d import InceptionI3d
    print("Import InceptionI3d từ dataset của bạn thành công!")
except ImportError as e:
    print(f"LỖI: Không thể import I3D. Kiểm tra lại đường dẫn: {e}")
    print(f"Đảm bảo file 'pytorch_i3d.py' nằm trong dataset '{I3D_SOURCE_DIR}'")

In [ ]:
# --- 1. ĐƯỜNG DẪN ĐẦU VÀO ---

# !!! Sửa 'your-video-dataset-name' thành tên dataset video của bạn
RAW_VIDEO_DIR = Path("/kaggle/input/rawvideo/RawVideo")

# !!! Sửa 'my-i3d-files' thành tên dataset I3D của bạn
I3D_FILES_DATASET_DIR = Path("/kaggle/input/my-i3d-files")
MODEL_WEIGHTS_PATH = I3D_FILES_DATASET_DIR / "rgb_imagenet.pt" 
# (Đảm bảo tên tệp chính xác là 'rgb_imagenet.pt')

# --- 2. ĐƯỜNG DẪN ĐẦU RA ---
FEATURE_DIR = Path("/kaggle/working/Features_I3D_10crop")

# --- 3. CẤU HÌNH TRÍCH XUẤT ---
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
SNIPPET_LENGTH = 16
INPUT_SIZE = 224

# --- MỚI: DANH SÁCH THƯ MỤC CẦN XỬ LÝ ---
# FOLDER_LIST = [
#     "", "Robbery", 
#     "Shooting", "Shoplifting", "Stealing", "Vandalism",
#     "Normal"  # Đảm bảo bạn có cả thư mục Normal
# ]
FOLDER_LIST = [
    "Shoplifting"
]
# --- MỚI: CẤU HÌNH RESUMABLE ---
# File .npy nhỏ hơn 1KB (1024 bytes) sẽ bị coi là lỗi và trích xuất lại
MIN_VALID_SIZE_BYTES = 1024 

print(f"Sử dụng thiết bị: {DEVICE}")
print(f"Thư mục Video nguồn: {RAW_VIDEO_DIR}")
print(f"Thư mục Features đích: {FEATURE_DIR}")
print(f"Đường dẫn trọng số: {MODEL_WEIGHTS_PATH}")
print(f"Sẽ xử lý {len(FOLDER_LIST)} thư mục đã định nghĩa.")

In [ ]:
def load_i3d_model(weights_path):
    print("Đang tải mô hình I3D...")
    # Khởi tạo mô hình I3D cho 400 lớp (Kinetics)
    model = InceptionI3d(400, in_channels=3)
    
    # Tải trọng số
    model.load_state_dict(torch.load(weights_path))
    model.to(DEVICE)
    model.eval()  # Chuyển sang chế độ đánh giá (rất quan trọng)
    
    # Đăng ký hook để lấy output từ lớp 'avg_pool'
    # Đây là đặc trưng 2048-dim
    features = {}
    def get_features(name):
        def hook(model, input, output):
            # Output có shape (batch, 2048, 1, 1, 1)
            # Squeeze nó về (batch, 2048)
            features[name] = output.squeeze().cpu().data.numpy()
        return hook
    
    # Tên 'avg_pool' là tên lớp trong file pytorch_i3d.py
    model.avg_pool.register_forward_hook(get_features('avg_pool'))
    
    print("Tải mô hình I3D và hook thành công.")
    return model, features

# Tải mô hình
i3d_model, i3d_features_hook = load_i3d_model(MODEL_WEIGHTS_PATH)

In [ ]:
# I3D yêu cầu chuẩn hóa [-1, 1], nhưng chúng ta sẽ làm thủ công
# Xóa i3d_transform cũ:
# i3d_transform = transforms.Compose([
#     transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
# ])

def get_10_crop_snippets(snippet_frames):
    """
    Input: snippet_frames (16, H, W, 3) - mảng numpy
    Output: torch.Tensor (10, 3, 16, 224, 224) - Tensor đã chuẩn hóa
    """
    target_size = (INPUT_SIZE, INPUT_SIZE) # (224, 224)
    
    # Chuyển (T, H, W, C) -> (T, C, H, W)
    snippet_tensor = torch.from_numpy(snippet_frames).permute(0, 3, 1, 2).float()
    
    # Chuẩn hóa 0-1
    snippet_tensor /= 255.0
    
    # --- Tạo 10 crop ---
    # 1. Resize về kích thước lớn hơn một chút (ví dụ 256)
    # Kích thước (T, C, H_resized, W_resized)
    resized_snippet = transforms.functional.resize(snippet_tensor, (256, 340))
    
    # 2. Lấy 5 crop (TopLeft, TopRight, BottomLeft, BottomRight, Center)
    # transforms.functional.five_crop trả về một tuple (5, T, C, 224, 224)
    five_crops = transforms.functional.five_crop(resized_snippet, target_size)
    
    # 3. Lấy 5 crop lật (Flipped)
    flipped_snippet = transforms.functional.hflip(resized_snippet)
    five_crops_flipped = transforms.functional.five_crop(flipped_snippet, target_size)
    
    # Ghép 10 crop lại (10, T, C, 224, 224)
    # (T ở đây là 16)
    ten_crops = torch.stack(five_crops + five_crops_flipped)
    
    # 4. Chuẩn hóa I3D [-1, 1]
    # (T, C, H, W) -> (C, T, H, W) 
    ten_crops = ten_crops.permute(0, 2, 1, 3, 4) # (10, C, T, 224, 224)
    
    # --- PHẦN SỬA LỖI ---
    # Xóa vòng lặp 'for' và hàm 'i3d_transform'
    # Áp dụng chuẩn hóa (0, 1) -> (-1, 1) cho toàn bộ tensor
    # Phép toán này tương đương với Normalize(mean=0.5, std=0.5)
    normalized_ten_crops = (ten_crops * 2.0) - 1.0
    # --- KẾT THÚC PHẦN SỬA LỖI ---
        
    return normalized_ten_crops

In [ ]:
def extract_video_features(video_path):
    try:
        vr = VideoReader(str(video_path), ctx=cpu(0))
        num_frames = len(vr)
        frame_indices = list(range(num_frames))
        
        all_snippet_features = []
        
        # Lấy chỉ số cho các snippet 16-frame (không chồng lấn)
        for i in range(0, num_frames, SNIPPET_LENGTH):
            snippet_indices = frame_indices[i : i + SNIPPET_LENGTH]
            
            # Nếu snippet cuối < 16 frame, lặp lại frame cuối
            if len(snippet_indices) < SNIPPET_LENGTH:
                last_frame_index = snippet_indices[-1]
                padding = [last_frame_index] * (SNIPPET_LENGTH - len(snippet_indices))
                snippet_indices.extend(padding)
                
            # Đọc 16 frame
            # get_batch trả về (T, H, W, C)
            snippet_frames = vr.get_batch(snippet_indices).asnumpy()
            
            # Xử lý 10-crop
            # -> (10, 3, 16, 224, 224)
            input_tensor = get_10_crop_snippets(snippet_frames)
            
            # Đưa qua I3D
            with torch.no_grad():
                i3d_model(input_tensor.to(DEVICE))
            
            # Lấy đặc trưng từ hook
            # -> (10, 2048)
            features = i3d_features_hook['avg_pool'].copy()
            
            # features shape là (10, 2048)
            all_snippet_features.append(features)

        if not all_snippet_features:
            print(f"  !!! Cảnh báo: Video {video_path.name} quá ngắn hoặc không đọc được, bỏ qua.")
            return None # Video quá ngắn hoặc không đọc được
            
        # Ghép tất cả các snippet lại
        # -> (T, 10, 2048)
        final_video_features = np.stack(all_snippet_features)
        return final_video_features

    except Exception as e:
        print(f"  !!! LỖI khi xử lý {video_path.name}: {e}")
        return None

In [ ]:
print("--- BẮT ĐẦU TRÍCH XUẤT ĐẶC TRƯNG ---")

# Các định dạng video
video_extensions = ["*.mp4", "*.avi", "*.mkv"]

total_videos = 0
processed_videos = 0
skipped_videos = 0
error_videos = 0

# --- THAY ĐỔI: Dùng FOLDER_LIST đã định nghĩa ---
for category_name in FOLDER_LIST:
    category_path = RAW_VIDEO_DIR / category_name
    
    if not category_path.is_dir():
        print(f"\nCảnh báo: Không tìm thấy thư mục '{category_name}', bỏ qua.")
        continue

    print(f"\nĐang xử lý thư mục: {category_name}")
    
    # Tạo thư mục đích (giữ nguyên cấu trúc)
    dest_category_path = FEATURE_DIR / category_name
    dest_category_path.mkdir(parents=True, exist_ok=True)
    
    video_files = []
    for ext in video_extensions:
        video_files.extend(list(category_path.glob(ext)))
    
    if not video_files:
        print(f"  Không tìm thấy video nào.")
        continue
        
    for video_path in tqdm(video_files, desc=f"  {category_name}"):
        total_videos += 1
        video_name = video_path.stem
        npy_save_path = dest_category_path / f"{video_name}.npy"
        
        # --- THAY ĐỔI: Kiểm tra Resumable nâng cao ---
        if npy_save_path.exists():
            try:
                file_size = npy_save_path.stat().st_size
                if file_size > MIN_VALID_SIZE_BYTES:
                    # File tồn tại và hợp lệ, bỏ qua
                    skipped_videos += 1
                    continue
                else:
                    # File quá nhỏ (lỗi), trích xuất lại
                    print(f"\n  Tệp {npy_save_path.name} bị lỗi (size: {file_size}B), đang trích xuất lại...")
            except OSError as e:
                # Không đọc được file (ví dụ: lỗi ổ đĩa), trích xuất lại
                print(f"\n  Không thể đọc tệp {npy_save_path.name} ({e}), đang trích xuất lại...")
        # --- KẾT THÚC KIỂM TRA ---

        # Trích xuất
        features = extract_video_features(video_path)
        
        # Lưu file .npy
        if features is not None:
            np.save(npy_save_path, features)
            processed_videos += 1
        else:
            # Lỗi từ hàm extract_video_features
            error_videos += 1

print("\n--- TRÍCH XUẤT HOÀN TẤT ---")
print(f"Tổng số video đã kiểm tra: {total_videos}")
print(f"Đã xử lý (mới): {processed_videos} videos")
print(f"Đã bỏ qua (đã có và hợp lệ): {skipped_videos} videos")
print(f"Lỗi (không xử lý được): {error_videos} videos")
print(f"Các tệp .npy đã được lưu vào: {FEATURE_DIR}")

In [ ]:
import os
import zipfile

OUTPUT_DIR = Path("/kaggle/working/Features_I3D_10crop")
ZIP_PATH = Path("/kaggle/working/Features_I3D_10crop.zip")

def zip_directory(folder_path, zip_path):
    print(f"Bắt đầu nén thư mục: {folder_path} -> {zip_path}")
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                # Tạo đường dẫn tương đối để giữ cấu trúc thư mục
                relative_path = os.path.relpath(os.path.join(root, file), folder_path)
                zipf.write(os.path.join(root, file), arcname=relative_path)
    print("Nén hoàn tất!")

# Chỉ chạy nén nếu thư mục features tồn tại
if OUTPUT_DIR.is_dir():
    zip_directory(OUTPUT_DIR, ZIP_PATH)
else:
    print(f"Không tìm thấy thư mục {OUTPUT_DIR} để nén.")

In [ ]:
# !rm -rf /kaggle/working/name.file
